In [ ]:
pip install newspaper3k

     |████████████████████████████████| 215kB 7.0MB/s 
     |████████████████████████████████| 92kB 7.6MB/s 
     |████████████████████████████████| 81kB 8.6MB/s 
     |████████████████████████████████| 7.4MB 32.8MB/s 
  Created wheel for feedfinder2: filename=feedfinder2-0.0.4-cp36-none-any.whl size=3358 sha256=d6fed33533d90aaa68af4623756351a51ccd541a19a440c645a6aa6be64f257e
  Stored in directory: /root/.cache/pip/wheels/de/03/ca/778e3a7a627e3d98836cc890e7cb40c7575424cfd3340f40ed
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-cp36-none-any.whl size=13538 sha256=84b7e73f6b69b8a26ab2201c683e05c7bf36a8d7df491168be65068eece9e5c2
  Stored in directory: /root/.cache/pip/wheels/81/2b/43/a02ede72324dd40cdd7ca53aad718c7710628e91b8b0dc0f02
  Created wheel for jieba3k: filename=jieba3k-0.35.1-cp36-none-any.whl size=7398406 sha256=600b5043742ff664f369041cdb3067a188a613c606bdb55e5fe5e88d3212e686
  Stored in directory: /root/.cache/pip/wheels/83/15/9c/a3f1f67e7f7181170ad37d32e503c35d

In [ ]:
from newspaper import Article
import newspaper
import nltk
nltk.download('punkt')
kompas_paper = newspaper.build('https://kompas.com/')

for article in kompas_paper.articles:
    print(article.url) 
    for category in kompas_paper.category_urls():
          print(category)
          kompas_article = kompas_paper.articles[0]
          kompas_article.download()
          kompas_article.parse()
          kompas_article.nlp()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
https://tekno.kompas.com/read/2020/12/28/20061677/miliarder-pengembang-game-asal-china-meninggal-akibat-diracun
https://kompas.com/
https://money.kompas.com
https://indeks.kompas.com
https://foto.kompas.com
https://inside.kompas.com
https://internasional.kompas.com
https://kilasbadannegara.kompas.com
https://sains.kompas.com
https://www.kompas.id
https://sorotpolitik.kompas.com
https://bola.kompas.com
https://health.kompas.com
https://lifestyle.kompas.com
https://balikpapan.kompas.com
https://www.kompas.com
https://palembang.kompas.com
https://kilasbumn.kompas.com
https://regional.kompas.com
https://sorot.kompas.com
https://inspirasli.kompas.com
http://kolom.kompas.com
http://www.kompas.tv
https://jeo.kompas.com
https://properti.kompas.com
https://superapps.kompas.com
http://indeks.kompas.com
https://nasional.kompas.com
https://surabaya.kompas.com
https://travel.kompas.com
https://

In [ ]:
import re
import string
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import TfidfVectorizer

def retrieve_docs_and_clean():
  # Untuk mendapatkan link berita populer
  r = requests.get('https://bola.kompas.com/')
  soup = BeautifulSoup(r.content, 'html.parser')

  link = []
  for i in soup.find('div', {'class':'most__wrap clearfix'}).find_all('a'):
      i['href'] = i['href'] + '?page=all'
      link.append(i['href'])

  # Retrieve Paragraphs
  documents = []
  for i in link:
      r = requests.get(i)
      soup = BeautifulSoup(r.content, 'html.parser')

      sen = []
      for i in soup.find('div', {'class':'read__content'}).find_all('p'):
          sen.append(i.text)
      documents.append(' '.join(sen))

  # Clean Paragraphs
  documents_clean = []
  for d in documents:
      document_test = re.sub(r'[^\x00-\x7F]+', ' ', d)
      document_test = re.sub(r'@\w+', '', document_test)
      document_test = document_test.lower()
      document_test = re.sub(r'[%s]' % re.escape(string.punctuation), ' ', document_test)
      document_test = re.sub(r'[0-9]', '', document_test)
      document_test = re.sub(r'\s{2,}', ' ', document_test)
      documents_clean.append(document_test)

  return documents_clean

In [ ]:
docs = retrieve_docs_and_clean()
# Create Term-Document Matrix with TF-IDF weighting
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(docs)

# Create a DataFrame
df = pd.DataFrame(X.T.toarray(), index=vectorizer.get_feature_names())
df.head()

,0,1,2,3,4,5,6,7,8,9
abad,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.185876,0.000000,0.000000
about,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.046469,0.000000,0.000000
acara,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.046469,0.000000,0.000000
ada,0.000000,0.027549,0.027518,0.078508,0.000000,0.000000,0.000000,0.000000,0.000000,0.038908
adalah,0.053494,0.036990,0.073898,0.000000,0.016539,0.019458,0.011828,0.020628,0.045877,0.000000


In [ ]:
def get_similar_articles(q, df):
  print("query:", q)
  print("Berikut artikel dengan nilai cosine similarity tertinggi: ")
  q = [q]
  q_vec = vectorizer.transform(q).toarray().reshape(df.shape[0],)
  sim = {}
  for i in range(10):
    sim[i] = np.dot(df.loc[:, i].values, q_vec) / np.linalg.norm(df.loc[:, i]) * np.linalg.norm(q_vec)
  
  sim_sorted = sorted(sim.items(), key=lambda x: x[1], reverse=True)
  
  for k, v in sim_sorted:
    if v != 0.0:
      print("Nilai Similaritas:", v)
      print(docs[k])
      print()


q1 = 'barcelona'
q2 = 'gareth bale'
q3 = 'shin tae yong'
q4 = 'indonesia'
q5 = 'real madrid'
q6 = 'liga inggris'
q7 = 'liga indondesia'
q8 = 'messi'
q9 = 'moh salah'
q10 = 'pemain terbaik'

get_similar_articles(q1, df)
print('-'*100)
get_similar_articles(q2, df)
print('-'*100)
get_similar_articles(q3, df)
get_similar_articles(q4, df)
print('-'*100)
get_similar_articles(q5, df)
print('-'*100)
get_similar_articles(q6, df)
get_similar_articles(q7, df)
print('-'*100)
get_similar_articles(q8, df)
print('-'*100)
get_similar_articles(q9, df)
print('-'*100)
get_similar_articles(q10, df)
print('-'*100)


query: barcelona
Berikut artikel dengan nilai cosine similarity tertinggi: 
Nilai Similaritas: 0.3164450276599631
 barcelona kompas com kapten barcelona lionel messi masih sakit hati saat mengingat transfer luis suarez ke atletico madrid awal musim ini secara spesifik lionel messi mengkritik cara barcelona memperlakukan luis suarez luis suarez terpaksa pindah ke atletico madrid setelah dirinya tidak masuk ke dalam rencana pelatih anyar barcelona ronald koeman pada awal kedatangannya ronald koeman secara terbuka memang sudah menyatakan ingin melakukan peremajaan skuad di barcelona tidak main main ronald koeman yang terkenal tegas langsung mendepak beberapa pemain senior barcelona beberapa hari setelah diresmikan baca juga luis suarez ungkap hal yang bisa bikin lionel messi bertahan di barcelona luis suarez yang menjadi salah satu korban perombakan skuad ala ronald koeman bisa dikatakan didepak barcelona secara tidak hormat sebab luis suarez yang masih memiliki sisa kontrak satu tahun di

In [ ]:
from newspaper import Article
url = 'https://www.kompas.com/sports/read/2020/12/28/08000028/cristiano-ronaldo-pemain-terbaik-abad-21-kalahkan-messi-dan-mo-salah?_ga=2.156083451.176857041.1609131003-1130194177.1573745978'
article = Article(url)
article.download()
article.parse()
article.text

'KOMPAS.com - Bintang Juventus, Cristiano Ronaldo, dinobatkan sebagai pemain terbaik abad ke-21 dalam acara Globe Soccer Awards 2020.\n\nMalam penganugerahaan diadakan di Dubai, Uni Emirat Arab, Senin (28/12/2020) dini hari WIB.\n\nCristiano Ronaldo berhak atas penghargaan tersebut setelah mengalahkan Lionel Messi (Barcelona) dan Mohamed Salah (Liverpool).\n\nPemain asal Portugal itu diakui sebagai pemain terbaik periode 2001-2020 berkat serangkaian prestasinya baik bersama klub maupun tim nasional.\n\nBaca juga: Sebut Milan seperti Rumah, Sinyal Ibrahimovic Bertahan Lebih Lama di San Siro?\n\n???? Congratulations to ???????? CRISTIANO RONALDO on winning the ?Globe Soccer Award for the? PLAYER OF THE CENTURY 2001-2020? ???? #CristianoRonaldo #CR7 #Bvlgari #GlobeSoccer pic.twitter.com/Y8Bg2ZfapO\n\nDalam rentang waktu tersebut, Ronaldo meraih banyak trofi termasuk lima Liga Champions, tiga Liga Inggris, dua Liga Spanyol, dua Liga Italia, serta juara Euro 2016 dan UEFA Nations League 201

In [ ]:
article.authors

['Kompas Cyber Media', 'Farahdilla Puspa', 'Afp', 'Marco Bertorello']

In [ ]:
article.publish_date

datetime.datetime(2020, 12, 28, 0, 0)

In [ ]:
article.nlp()
article.keywords
article.summary

'KOMPAS.com - Bintang Juventus, Cristiano Ronaldo, dinobatkan sebagai pemain terbaik abad ke-21 dalam acara Globe Soccer Awards 2020.\nCristiano Ronaldo berhak atas penghargaan tersebut setelah mengalahkan Lionel Messi (Barcelona) dan Mohamed Salah (Liverpool).\nCRISTIANO RONALDO on winning the ?Globe Soccer Award for the?\n"Ini pencapaian yang luar biasa dan memberi saya motivasi untuk terus maju," kata Cristiano Ronaldo dilansir dari Marca.\n"Dinobatkan sebagai yang terbaik adalah suatu kehormatan besar.'

In [ ]:
pip install autocorrect

     |████████████████████████████████| 624kB 5.3MB/s 
  Created wheel for autocorrect: filename=autocorrect-2.2.2-cp36-none-any.whl size=621492 sha256=c51813acb64452f7a2c1e7a09406f952a1b45112a96e30e290dedfe3abd6cf3d
  Stored in directory: /root/.cache/pip/wheels/b4/0b/7d/98268d64c8697425f712c897265394486542141bbe4de319d6
Successfully built autocorrect


In [ ]:
pip install pyspellchecker

     |████████████████████████████████| 1.9MB 4.9MB/s 


In [ ]:
from spellchecker import SpellChecker

spell = SpellChecker()
words = spell.split_words("Bintang Juventus, Cristiano Ronaldo, dinbatkan sebgi pemain terbaik abad ke-21 dalam acara Globe Soccer Awards 2020.\nCristiano Ronaldo berhak atas penghargaan tersebut setelah mengalahkan Lionel Messi (Barcelona) dan Mohamed Salah (Liverpool).\nCRISTIANO RONALDO on winning the ?Globe Soccer Award for the?\n Ini pencapaian yang luar biasa dan memberi saya motivasi untuk terus maju, kata Cristiano Ronaldo dilansir dari Marca.\n Dinobatkan sebagai yang terbaik adalah suatu kehormatan besar.")
test = [spell.correction(word) for word in words]